In [17]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import scipy as sp
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.random import choice
from tree.tree import DecisionTree, RandomForest
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
%aimport tree

In [19]:
%aimport

Modules to reload:
tree

Modules to skip:



# Spam

In [20]:
spam = scipy.io.loadmat("spam-dataset/spam_data.mat")

In [21]:
spam_raw_train_X = spam['training_data']
spam_raw_train_y = spam['training_labels'].reshape((5172,))
spam_raw_test_X = spam['test_data']
shuff = choice(len(spam_raw_train_X), len(spam_raw_train_X), False)
spam_raw_train_X = spam['training_data'][shuff]
spam_raw_train_y = spam['training_labels'].reshape((5172,))[shuff]
spam_raw_test_X = spam['test_data']

In [96]:
a1, b1, at, bt = train_test_split(spam_raw_train_X, spam_raw_train_y, test_size=0.65)

In [97]:
print(len(a1), len(b1))

1810 3362


## Spam Decision Tree

In [103]:
%autoreload
dt = DecisionTree({
        "max_depth":50,
        "min_points": 1
    })
dt.train(a1, at)
scored = dt.score(b1, bt)
print(sum(scored)/len(scored))
# 0.74317

0.783759666865


In [104]:
spam_out = pd.DataFrame(dt.predict(spam_raw_test_X)).reset_index()
spam_out.head()

,index,0
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0


In [105]:
spam_out.columns = ['Id', 'Category']
spam_out.Id = spam_out.Id + 1
spam_out.to_csv("spam_prediction.csv", index=False)

## Spam Random Forest

In [111]:
# Kaggle: 0.78108

In [125]:
%autoreload
rf = RandomForest({
        'ntrees': 50,
        },{
        "max_depth": 8,
        "min_points": 1,
        'random_subset': True,
        'subset_size': 10
        })
print(rf)
rf.train(a1, at)
scored = rf.score(b1, bt)
#print(pd.DataFrame([scored, bt]).T)
print(sum(scored)/len(scored))
# scored = rf.score(a1, at)
# print(sum(scored)/len(scored))

Random Forest. 50 Trees
trained all trees
0.795657346817


In [126]:
spam_out = pd.DataFrame(rf.predict(spam_raw_test_X)).reset_index()
spam_out.head()
spam_out.columns = ['Id', 'Category']
spam_out.Id = spam_out.Id + 1
spam_out.to_csv("rf_spam_prediction.csv", index=False)

# Census

In [155]:
census = pd.read_csv("census-dataset/data.csv", na_values=["?"])
census_test = pd.read_csv("census-dataset/test_data.csv", na_values=["?"])

In [156]:
categoricals = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
non_categoricals = list(set(census.columns).difference(set(categoricals)).difference(set(["label"])))

In [157]:
census.shape

(32724, 15)

In [158]:
census_X = np.hstack([census[non_categoricals].values, pd.get_dummies(census[categoricals]).values])
census_y = census.label.values
census_test_X = np.hstack([census_test[non_categoricals].values, pd.get_dummies(census_test[categoricals]).values])
print(census_X.shape, census_y.shape)
c1, c21, c1t, c2t = train_test_split(spam_raw_train_X, spam_raw_train_y, test_size=.65)

(32724, 105) (32724,)


## Census Decision Tree

In [159]:
%autoreload
dt2 = DecisionTree({
        "max_depth":1000,
        "min_points": 15
    })
dt2.train(c1, c1t)
scored = dt2.score(c21, c2t)
print(sum(scored)/len(scored))

0.776621058894


In [160]:
census_out = pd.DataFrame(dt2.predict(census_test_X)).reset_index()
print(census_out.head())
print(len(census_out))
census_out.columns = ['Id', 'Category']
census_out.Id = census_out.Id + 1
census_out.to_csv("dt_census_prediction.csv", index=False)

   index  0
0      0  0
1      1  0
2      2  0
3      3  0
4      4  0
16118


## Census Random Forest

In [161]:
%autoreload
rf2 = RandomForest({
        'ntrees': 50,
        },{
        "max_depth": 20,
        "min_points": 20,
        'random_subset': True,
        'subset_size': 30
        })
print(rf)
rf2.train(c1, c1t)
scored = rf2.score(c21, c2t)
#print(pd.DataFrame([scored, bt]).T)
print(sum(scored)/len(scored))
# scored = rf.score(a1, at)
# print(sum(scored)/len(scored))

Random Forest. 50 Trees
trained all trees
0.781380130874


In [162]:
census_out = pd.DataFrame(rf2.predict(census_test_X)).reset_index()
print(census_out.head())
print(len(census_out))
census_out.columns = ['Id', 'Category']
census_out.Id = census_out.Id + 1
census_out.to_csv("rf_census_prediction.csv", index=False)

   index  0
0      0  0
1      1  0
2      2  0
3      3  0
4      4  0
16118
